<a href="https://colab.research.google.com/github/mihir1125/machine-learning/blob/master/neural_networks_mnist_digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q mediapy

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import mediapy as media

In [18]:
#@title Helper functions
def label_to_vector(num):
    arr = np.zeros(10, dtype=np.uint8)
    arr[num] = 1
    return arr

In [3]:
# Fetch the dataset

mnist = tfds.image.MNIST()

In [4]:
# Describe the dataset with DatasetInfo

print(mnist.info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir=PosixGPath('/tmp/tmpyvtsb3twtfds'),
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)


In [5]:
# Download the data, prepare it, and write it to disk
mnist.download_and_prepare()

# Load data from disk as tf.data.Datasets
datasets = mnist.as_dataset()
train_ds, test_ds = datasets['train'], datasets['test']

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [34]:
X_train = np.array([example['image'].reshape(-1) for example in tfds.as_numpy(train_ds)])
y_train = np.array([label_to_vector(example['label']) for example in tfds.as_numpy(train_ds)])

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(28*28, )),
    tf.keras.layers.Dense(49, activation='sigmoid', name='Layer1'),
    tf.keras.layers.Dense(25, activation='sigmoid', name='Layer2'),
    tf.keras.layers.Dense(10, activation='sigmoid', name='Layer3')
], name='my_first_neural_network')

model.summary()

Model: "my_first_neural_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 49)                38465     
                                                                 
 Layer2 (Dense)              (None, 25)                1250      
                                                                 
 Layer3 (Dense)              (None, 10)                260       
                                                                 
Total params: 39,975
Trainable params: 39,975
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(0.001)
)

model.fit(X_train, y_train, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4343
Epoch 2/4
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3992
Epoch 3/4
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3712
Epoch 4/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3648


In [41]:
# Test
X_test = np.array([example['image'].reshape(-1) for example in tfds.as_numpy(test_ds)])
y_test = np.array([label_to_vector(example['label']) for example in tfds.as_numpy(test_ds)])

In [42]:
eval = model.evaluate(X_test, y_test)
print(eval)

313/313 [==============================] - 1s 2ms/step - loss: 0.3684
0.36839643120765686


In [46]:
n = np.random.randint(0, X_test.shape[0]-1)
media.show_image(X_test[n].reshape(28, 28, 1))
print(X_test[n].shape)
y_hat = model.predict(X_test[n].reshape(1, -1))
y = y_test[n]
print(f"Actual: {y.argmax()}\nPrediction: {y_hat.argmax()}")

""


(784,)
1/1 [==============================] - 0s 30ms/step
Actual: 8
Prediction: 7
